# Oculométrie et attention visuelle

Bienvenu dans ce notebook dédié à l'étude des données d'oculométrie et aux modèles d'attention visuelle.

## Un peu d'intuition

Exécutez le code ci-dessous qui affiche simplement les différentes images (stimuli) qui ont été visualisées par des observateurs / fournies en entrée de modèles d'attention.

In [ ]:
%matplotlib notebook

import warnings
import matplotlib.pyplot as plt
import skimage.io as io

for i in range(1,12):
    img=io.imread("Stimuli/img"+str(i)+".jpg")
    plt.figure()
    plt.imshow(img)

**Question 1:** A votre avis, quels sont les élements de chaque image qui vont attirer le regard des observateurs ?

**Réponse 1:**

## Observons la vérité terrain

Le script suivant affiche en plus des images (stimuli) les cartes de saillance construites à partir des données oculométriques des observateurs.

In [ ]:
for i in range(1,12):
    img=io.imread("Stimuli/img"+str(i)+".jpg")
    sal=io.imread("GroundTruthMaps/gt_sal"+str(i)+".jpg")
    plt.figure()
    plt.subplot(121) # 121 = 1 ligne, 2 colonnes, 1ère sous figure
    plt.imshow(img)
    plt.subplot(122) # 121 = 1 ligne, 2 colonnes, 1ère sous figure
    plt.imshow(sal)

**Question 2:** Quels sont les éléments / caractéristique d'image qui attirent le regard des observateurs ? Est-ce que ce ces observations confirment vos intutions de la question 1 ?

**Réponse 2:**

## Observation des performance d'un modèle d'attention

Le script ci-dessous permet de mesure la performance d'un modèle d'attention en utilisant un certain nombre de mesure (Corrélation, AUC, etc.).

In [ ]:
import numpy as np
import skimage.transform as tr
import sys
from pysaliency.benchmark import metrics

model_name="center-model"

# {Metric name:
#   func,
#   passes: should we compute 1vs2 and 2vs1 then average?
#   type: requires fixation or saliency maps as comparison
# }
metrics = {
    "AUC_Judd": [metrics.AUC_Judd, 1, 'fix'], # Binary fixation map
    "AUC_Borji": [metrics.AUC_Borji, 1, 'fix'], # Binary fixation map
    "NSS": [metrics.NSS, 1, 'fix'], # Binary fixation map
    "CC": [metrics.CC, 1, 'sal'], # Saliency map
    "SIM": [metrics.SIM, 1, 'sal'], # Saliency map
    "KLD": [metrics.kld, 2, 'sal'] } # Saliency map

M = {m:[] for m in metrics.keys()}

print("Processing",end="")
for im_number in range(1,12):

    print(".",end="")

    # Load fixation and saliency maps from ground truth

    sal_gt = { 
        "fix": np.transpose(np.load("./GroundTruthData/I" + str(im_number) + "_C0.npz")["arr_0"]),
        "sal": np.transpose(np.load("./GroundTruthData/I" + str(im_number) + "_C0_saliency.npz")["arr_0"])
        }

    (height, width) = sal_gt["sal"].shape

    # Load saliency maps from model
    sal_model = io.imread("./SalModelsOutputs/"+ model_name + "/sal_map_img" + str(im_number) + ".jpg" )
    sal_model = tr.resize(sal_model, (height, width))

    # Normalize data and resize models output
    sal_model = np.divide(sal_model, np.sum(sal_model))
    sal_gt["sal"] = np.divide(sal_gt["sal"], np.sum(sal_gt["sal"]))

    for metric in metrics.keys():
        func = metrics[metric]
        if func[1] == 2:
            m  = func[0](sal_model, sal_gt[func[2]])
            m += func[0](sal_gt['sal'], sal_model)
            m /= 2
        else:
            m = func[0](sal_model, sal_gt[func[2]])

        M[metric].append(m)
      
print("\nPer image results\n")
for kk, vv in M.items():
    print("{:10} : {}".format(kk, ["{: .2f}".format(number) for number in vv]))
    
print("\nMean/StdDev\n")
for kk, vv in M.items():
    print("{:10} : {:.2f} / {:.2f}".format(kk,np.mean(vv), np.std(vv)))

Chaque (groupe d') étudiant travaillera sur un modèle particulier. Veuillez adapter la variable `model_name` au nom du modèle que vous étudiez.

**Question 3:** Le modèle a-t-il les mêmes performances sur toutes les images ? Essayez d'expliquer pourquoi le modèle se comporte moins bien dans certaines situations.

**Réponse 3:**

**Question 4:** Les différentes mesures utilisée ont elles des résultats cohérents ? Quelles sont les conséquences pour l'analyse des performances d'un modèle ?

**Réponse 4:**

**Question 5:** Votre modèle a-t-il les mêmes performances sur toutes les images ? Quels sont les types / caractéristiques d'images sur lesquel le modèle fonctionne bien (ou mal) ?

**Réponse 5:**

**Question 6:** Comparez les performances de votre modèle avec celles des autres (étudiés par d'autres étudiants). Certains modèles sont ils plus performants que d'autres ? Lesquels ? Le sont ils sur toutes les images ?

**Réponse 6:**